In [1]:
import zipfile
import os

def unzip(zip_file_path, extract_dir):

    if not os.path.exists(extract_dir):
        os.makedirs(extract_dir)

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Extract all the contents
        zip_ref.extractall(extract_dir)

    print("Files extracted successfully!")

In [5]:
zip_file_path = '/content/drive/MyDrive/Luna/subset4.zip'
extract_dir = '/content/drive/MyDrive/Luna_Test/'
unzip(zip_file_path, extract_dir)

Files extracted successfully!


In [7]:
!pip install SimpleITK
# !pip install pandas==1.1.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 22.3 MB/s eta 0:00:00


In [8]:
# some directory for output the results:
OUTPUT_PATH = '/content/drive/MyDrive/Preprocessed_Luna_2/'

# Resource path which contains: annotations.csv, candidates.csv,
# and subdirectories containing .mhd files.
# This is the directory structure needed to run the code:
# (The code will use all .mhd and .raw files inside subdirectories which their name is in annotations or candidates)
'''
[RESOURCES_PATH]/
            annotations.csv
            candidates.csv
            subset0/
                        *.mhd
                        *.raw
            subset1/
                        *.mhd
                        *.raw
            my_custom_subset/
                        *.mhd
                        *.raw
'''
RESOURCES_PATH = '/content/drive/MyDrive/Luna_Test/'





PADDING_FOR_LOCALIZATION = 10
BLOCK_SIZE = 128
COORDS_CUBE_SIZE = 32
TARGET_SHAPE = (COORDS_CUBE_SIZE, COORDS_CUBE_SIZE, COORDS_CUBE_SIZE, 3, 5)
COORDS_SHAPE = (3, COORDS_CUBE_SIZE, COORDS_CUBE_SIZE, COORDS_CUBE_SIZE)
ANCHOR_SIZES = [10, 30, 60]
VAL_PCT = 0.2
TOTAL_EPOCHS = 100
DEFAULT_LR = 0.01

In [9]:
from functools import partial

import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage as ndi
import scipy
from skimage.filters import roberts
from skimage.measure import label, regionprops
from skimage.morphology import convex_hull_image, disk, binary_closing
from skimage.segmentation import clear_border


def argmax_3d(img: np.array):
    max1 = np.max(img, axis=0)
    argmax1 = np.argmax(img, axis=0)
    max2 = np.max(max1, axis=0)
    argmax2 = np.argmax(max1, axis=0)
    argmax3 = np.argmax(max2, axis=0)
    argmax_3d = (argmax1[argmax2[argmax3], argmax3], argmax2[argmax3], argmax3)
    return argmax_3d, img[argmax_3d]


def _get_cube_from_img_new(img, origin: tuple, block_size=128, pad_value=106.):
    assert 2 <= len(origin) <= 3
    final_image_shape = tuple([block_size] * len(origin))
    result = np.ones(final_image_shape) * pad_value
    start_at_original_images = []
    end_at_original_images = []
    start_at_result_images = []
    end_at_result_images = []
    for i, center_of_a_dim in enumerate(origin):
        start_at_original_image = int(center_of_a_dim - block_size / 2)
        end_at_original_image = start_at_original_image + block_size
        if start_at_original_image < 0:
            start_at_result_image = abs(start_at_original_image)
            start_at_original_image = 0
        else:
            start_at_result_image = 0
        if end_at_original_image > img.shape[i]:
            end_at_original_image = img.shape[i]
            end_at_result_image = start_at_result_image + (end_at_original_image - start_at_original_image)
        else:
            end_at_result_image = block_size
        start_at_original_images.append(start_at_original_image)
        end_at_original_images.append(end_at_original_image)
        start_at_result_images.append(start_at_result_image)
        end_at_result_images.append(end_at_result_image)
    # for simplicity
    sri = start_at_result_images
    eri = end_at_result_images
    soi = start_at_original_images
    eoi = end_at_original_images
    if len(origin) == 3:
        result[sri[0]:eri[0], sri[1]:eri[1], sri[2]:eri[2]] = img[soi[0]:eoi[0], soi[1]:eoi[1], soi[2]:eoi[2]]
    elif len(origin) == 2:
        result[sri[0]:eri[0], sri[1]:eri[1]] = img[soi[0]:eoi[0], soi[1]:eoi[1]]

    return result


def random_crop(img: np.array, centers: list, lungs_bounding_box: list, radii: list, main_nodule_idx: int,
                spacing: tuple,
                block_size: int,
                pad_value: float, margin: int):
    max_radius_index = np.max(np.round(radii[main_nodule_idx] / np.array(spacing)).astype(int))
    center_of_cube = list(centers[main_nodule_idx])
    shifts = []
    for i in range(len(centers[main_nodule_idx])):
        high = int(block_size / 2) - max_radius_index - margin
        if high < 0:
            print('negative high!!!')
            high = 0
        shift = np.random.randint(low=-abs(high), high=abs(high))
        center_of_cube[i] += shift
        shifts.append(shift)
    out_img = _get_cube_from_img_new(img, origin=tuple(center_of_cube), block_size=block_size, pad_value=pad_value)
    out_centers = []
    out_lungs_bounding_box = []
    for i in range(len(centers)):
        diff = np.array(centers[main_nodule_idx]) - np.array(centers[i])
        out_centers.append(
            tuple(np.array([int(block_size / 2)] * len(centers[i]), dtype=int) - np.array(shifts, dtype=int) - diff))
    for i in range(len(lungs_bounding_box)):
        diff = np.array(centers[main_nodule_idx]) - np.array(lungs_bounding_box[i])
        out_lungs_bounding_box.append(tuple(
            np.array([int(block_size / 2)] * len(lungs_bounding_box[i]), dtype=int) - np.array(shifts,
                                                                                               dtype=int) - diff))

    return out_img, out_centers, out_lungs_bounding_box


def _get_point_after_2d_rotation(in_points: list, shape: tuple, rot90s: int, flip: bool = False):
    assert len(in_points[0]) == 2 and len(shape) == 2
    rot90s = rot90s % 4
    result_points = []
    for in_point in in_points:
        result_point = list(in_point)
        for i in range(rot90s):
            previous = result_point.copy()
            axes = [0, 1]
            point_complement = (shape[0] - previous[0], shape[1] - previous[1])
            result_point[axes[0]] = point_complement[axes[1]]
            result_point[axes[1]] = previous[axes[0]]
        if flip:
            result_point[0] = shape[0] - result_point[0]
        result_points.append(tuple(result_point))
    return result_points


def _get_point_after_3d_rotation(in_points: list, shape: tuple, axes, rot90s: int, flip: bool = False):
    rot90s = rot90s % 4
    result_points = []
    for in_point in in_points:
        result_point = list(in_point)
        other_axis = [item for item in [0, 1, 2] if item not in axes]
        for i in range(rot90s):
            previous = result_point.copy()
            point_complement = np.array(shape, dtype=int) - np.array(previous, dtype=int)
            result_point[axes[0]] = point_complement[axes[1]]
            result_point[axes[1]] = previous[axes[0]]
        if flip:
            result_point[other_axis[0]] = shape[other_axis[0]] - result_point[other_axis[0]]
        result_points.append(tuple(result_point))
    return result_points


def rotate(img: np.array, spacing: tuple, centers: list, lungs_bounding_box: list, rotate_id: int):
    spacing = list(spacing)
    dimensions = len(img.shape)
    assert (dimensions == 3 and rotate_id < 24) or (dimensions == 2 and rotate_id < 8)
    other_axes = [i for i in range(dimensions)]

    if dimensions == 2:
        axis = [0]
        out_points = partial(_get_point_after_2d_rotation, shape=tuple(img.shape))
    else:  # dimensions == 3
        axis = rotate_id // 8
        other_axes.pop(axis)
        out_points = partial(_get_point_after_3d_rotation, shape=tuple(img.shape), axes=other_axes)

    which_rotation = rotate_id % 8
    flip = which_rotation >= 4
    rotation_times = (which_rotation % 4)

    spacing_exchanged = (which_rotation % 2) != 0
    if spacing_exchanged:
        if dimensions == 3:
            tmp = spacing[other_axes[0]]
            spacing[other_axes[0]] = spacing[other_axes[1]]
            spacing[other_axes[1]] = tmp
        elif dimensions == 2:
            tmp = spacing[0]
            spacing[0] = spacing[1]
            spacing[1] = tmp
    img = np.rot90(img, k=rotation_times, axes=other_axes)
    if flip:
        img = np.flip(img, axis=axis)
    return img, tuple(spacing), out_points(in_points=centers, rot90s=rotation_times, flip=flip), out_points(
        in_points=lungs_bounding_box, rot90s=rotation_times, flip=flip)


def scale(img: np.array, scale_factor: float, spacing: tuple, centers: list, lungs_bounding_box: list, radii: list):
    assert (.75 <= scale_factor <= 1.25)
    out_centers = [tuple(np.rint(np.array(c) * scale_factor).astype(int)) for c in centers]
    out_lungs_bounding_box = [tuple(np.rint(np.array(b) * scale_factor).astype(int)) for b in lungs_bounding_box]
    out_radii = [r * scale_factor for r in radii]
    spacing = np.array(spacing) * scale_factor
    img1 = scipy.ndimage.zoom(img, spacing, mode='nearest')
    return img1, tuple(spacing), out_centers, out_lungs_bounding_box, out_radii


def get_augmented_cube(img: np.array, radii: list, centers: list, main_nodule_idx: int, spacing: tuple,
                       lungs_bounding_box: list, block_size=128, pad_value=106, margin=10, rot_id=None):
    scale_factor = np.random.random() / 2 + .75
    rotate_id = np.random.randint(0, 24) if not rot_id else rot_id
    img1, spacing1, centers1, lungs_bounding_box1, radii1 = scale(img, scale_factor=scale_factor, spacing=spacing,
                                                                  centers=centers,
                                                                  lungs_bounding_box=lungs_bounding_box, radii=radii)
    img2, centers2, lungs_bounding_box2 = random_crop(img=img1, centers=centers1,
                                                      lungs_bounding_box=lungs_bounding_box1, radii=radii1,
                                                      main_nodule_idx=main_nodule_idx, spacing=spacing1,
                                                      block_size=block_size, pad_value=pad_value, margin=margin)
    existing_centers_in_patch = []
    for i in range(len(centers2)):
        dont_count = False
        for ax in centers2[i]:
            if not (0 <= ax <= block_size):
                dont_count = True
                break
        if not dont_count:
            existing_centers_in_patch.append(i)
    img3, spacing2, centers3, lungs_bounding_box3 = rotate(img=img2, spacing=spacing1, centers=centers2,
                                                           lungs_bounding_box=lungs_bounding_box2, rotate_id=rotate_id)
    return img3, radii1, centers3, lungs_bounding_box3, spacing2, existing_centers_in_patch


def get_segmented_lungs(im, plot=False):
    '''
    This funtion segments the lungs from the given 2D slice.
    '''
    plt_number = 0
    # Original image label: 0
    if plot:
        f, plots = plt.subplots(12, 1, figsize=(5, 40))
        plots[plt_number].axis('off')
        plots[plt_number].set_title(f'{plt_number}')
        plots[plt_number].imshow(im, cmap=plt.cm.bone)
        plt_number += 1

    # Step 1: Convert into a binary image.
    # image label: 1
    binary = im < -604
    if plot:
        plots[plt_number].axis('off')
        plots[plt_number].set_title(f'{plt_number}')
        plots[plt_number].imshow(binary, cmap=plt.cm.bone)
        plt_number += 1
    # Step 2: Remove the blobs connected to the border of the image.
    # image label: 2
    cleared = clear_border(binary)
    if plot:
        plots[plt_number].axis('off')
        plots[plt_number].set_title(f'{plt_number}')
        plots[plt_number].imshow(cleared, cmap=plt.cm.bone)
        plt_number += 1
    # Step 3: Label the image.
    # image label: 3
    label_image = label(cleared)
    if plot:
        plots[plt_number].axis('off')
        plots[plt_number].set_title(f'{plt_number}')
        plots[plt_number].imshow(label_image, cmap=plt.cm.bone)
        plt_number += 1

    # Step 4: Keep the labels with 2 largest areas and segment two lungs.
    # image label: 4
    areas = [r.area for r in regionprops(label_image)]
    areas.sort()
    labels = []
    if len(areas) > 2:
        for region in regionprops(label_image):
            if region.area < areas[-2]:
                for coordinates in region.coords:
                    label_image[coordinates[0], coordinates[1]] = 0
            else:
                coordinates = region.coords[0]
                labels.append(label_image[coordinates[0], coordinates[1]])
    else:
        labels = [1, 2]
    if plot:
        plots[plt_number].axis('off')
        plots[plt_number].set_title(f'{plt_number}')
        plots[plt_number].imshow(label_image, cmap=plt.cm.bone)
        plt_number += 1
    # Step 5: Fill in the small holes inside the mask of lungs which we seperate right and left lung.
    # r and l are symbolic and they can be actually left and right!
    # image labels: 5, 6
    rig = label_image == labels[0]
    lef = label_image == labels[1]
    r_edges = roberts(rig)
    l_edges = roberts(lef)
    rig = ndi.binary_fill_holes(r_edges)
    lef = ndi.binary_fill_holes(l_edges)
    if plot:
        plots[plt_number].axis('off')
        plots[plt_number].set_title(f'{plt_number}')
        plots[plt_number].imshow(rig, cmap=plt.cm.bone)
        plt_number += 1

        plots[plt_number].axis('off')
        plots[plt_number].set_title(f'{plt_number}')
        plots[plt_number].imshow(lef, cmap=plt.cm.bone)
        plt_number += 1

    # Step 6: convex hull of each lung
    # image labels: 7, 8
    rig = convex_hull_image(rig)
    lef = convex_hull_image(lef)
    if plot:
        plots[plt_number].axis('off')
        plots[plt_number].set_title(f'{plt_number}')
        plots[plt_number].imshow(rig, cmap=plt.cm.bone)
        plt_number += 1

        plots[plt_number].axis('off')
        plots[plt_number].set_title(f'{plt_number}')
        plots[plt_number].imshow(lef, cmap=plt.cm.bone)
        plt_number += 1
    # Step 7: joint two separated right and left lungs.
    # image label: 9
    sum_of_lr = rig + lef
    binary = sum_of_lr > 0
    if plot:
        plots[plt_number].axis('off')
        plots[plt_number].set_title(f'{plt_number}')
        plots[plt_number].imshow(binary, cmap=plt.cm.bone)
        plt_number += 1
    # Step 8: Closure operation with a disk of radius 10. This operation is
    # to keep nodules attached to the lung wall.
    # image label: 10
    selem = disk(10)
    binary = binary_closing(binary, selem)
    if plot:
        plots[plt_number].axis('off')
        plots[plt_number].set_title(f'{plt_number}')
        plots[plt_number].imshow(binary, cmap=plt.cm.bone)
        plt_number += 1
    # Step 9: Superimpose the binary mask on the input image.
    # image label: 11
    get_high_vals = binary == 0
    im[get_high_vals] = 0
    if plot:
        plots[plt_number].axis('off')
        plots[plt_number].set_title(f'{plt_number}')
        plots[plt_number].imshow(im, cmap=plt.cm.bone)
        plt_number += 1

    return im, convex_hull_image(binary)

In [10]:
import scipy.misc
import numpy as np
import SimpleITK as sitk
from glob import glob
from skimage.measure import regionprops


class CTScan(object):
    def __init__(self, seriesuid, centers, radii, clazz):
        self._seriesuid = seriesuid
        self._centers = centers
        paths = glob(f'''{RESOURCES_PATH}/*/{self._seriesuid}.mhd''')
        path = paths[0]
        self._ds = sitk.ReadImage(path)
        self._spacing = np.array(list(reversed(self._ds.GetSpacing())))
        self._origin = np.array(list(reversed(self._ds.GetOrigin())))
        self._image = sitk.GetArrayFromImage(self._ds)
        self._radii = radii
        self._clazz = clazz
        self._mask = None

    def preprocess(self):
        self._resample()
        self._segment_lung_from_ct_scan()
        self._normalize()
        self._zero_center()
        self._change_coords()

    def save_preprocessed_image(self):
        subdir = 'negatives' if self._clazz == 0 else 'positives'
        file_path = f'''preprocessed/{subdir}/{self._seriesuid}.npy'''
        np.save(f'{OUTPUT_PATH}/{file_path}', self._image)

    def get_info_dict(self):
        (min_z, min_y, min_x, max_z, max_y, max_x) = (None, None, None, None, None, None)
        for region in regionprops(self._mask):
            min_z, min_y, min_x, max_z, max_y, max_x = region.bbox
        assert (min_z, min_y, min_x, max_z, max_y, max_x) != (None, None, None, None, None, None)
        min_point = (min_z, min_y, min_x)
        max_point = (max_z, max_y, max_x)
        return {'seriesuid': self._seriesuid, 'radii': self._radii, 'centers': self._centers,
                'spacing': list(self._spacing), 'lungs_bounding_box': [min_point, max_point], 'class': self._clazz}

    def _resample(self):
        spacing = np.array(self._spacing, dtype=np.float32)
        new_spacing = [1, 1, 1]
        imgs = self._image
        new_shape = np.round(imgs.shape * spacing / new_spacing)
        true_spacing = spacing * imgs.shape / new_shape
        resize_factor = new_shape / imgs.shape
        imgs = scipy.ndimage.zoom(imgs, resize_factor, mode='nearest')
        self._image = imgs
        self._spacing = true_spacing

    def _segment_lung_from_ct_scan(self):
        result_img = []
        result_mask = []
        num = 0
        for slicee in self._image:
            rimg, rmsk = get_segmented_lungs(slicee)
            result_img.append(rimg)
            result_mask.append(rmsk)
        self._image = np.asarray(result_img)
        self._mask = np.asarray(result_mask, dtype=int)

    def _world_to_voxel(self, worldCoord):
        stretchedVoxelCoord = np.absolute(np.array(worldCoord) - np.array(self._origin))
        voxelCoord = stretchedVoxelCoord / np.array(self._spacing)
        return voxelCoord.astype(int)

    def _get_world_to_voxel_coords(self, idx):
        return tuple(self._world_to_voxel(self._centers[idx]))

    def _get_voxel_coords(self):
        voxel_coords = [self._get_world_to_voxel_coords(j) for j in range(len(self._centers))]
        return voxel_coords

    def _change_coords(self):
        new_coords = self._get_voxel_coords()
        self._centers = new_coords

    def _normalize(self):
        MIN_BOUND = -1200
        MAX_BOUND = 600.
        self._image = (self._image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
        self._image[self._image > 1] = 1.
        self._image[self._image < 0] = 0.
        self._image *= 255.

    def _zero_center(self):
        PIXEL_MEAN = 0.25 * 256
        self._image = self._image - PIXEL_MEAN


class PatchMaker(object):
    def __init__(self, seriesuid: str, coords: list, radii: list, spacing: list, lungs_bounding_box: list,
                 file_path: str,
                 clazz: int):
        self._seriesuid = seriesuid
        self._coords = coords
        self._spacing = spacing
        self._radii = radii
        self._image = np.load(file=f'{file_path}')
        self._clazz = clazz
        self._lungs_bounding_box = lungs_bounding_box

    def _get_augmented_patch(self, idx, rot_id=None):
        return get_augmented_cube(img=self._image, radii=self._radii, centers=self._coords,
                                  spacing=tuple(self._spacing), rot_id=rot_id, main_nodule_idx=idx,
                                  lungs_bounding_box=self._lungs_bounding_box)

    def get_augmented_patches(self):
        radii = self._radii
        list_of_dicts = []
        for i in range(len(self._coords)):
            times_to_sample = 1
            if radii[i] > 15.:
                times_to_sample = 2
            elif radii[i] > 20.:
                times_to_sample = 6
            for j in range(times_to_sample):
                rot_id = int((j / times_to_sample) * 24 + np.random.randint(0, int(24 / times_to_sample)))
                img, radii2, centers, lungs_bounding_box, spacing, existing_nodules_in_patch = \
                    self._get_augmented_patch(idx=i, rot_id=rot_id)
                existing_radii = [radii2[i] for i in existing_nodules_in_patch]
                existing_centers = [centers[i] for i in existing_nodules_in_patch]
                subdir = 'negatives' if self._clazz == 0 else 'positives'
                file_path = f'''augmented/{subdir}/{self._seriesuid}_{i}_{j}.npy'''
                list_of_dicts.append(
                    {'seriesuid': self._seriesuid, 'centers': existing_centers, 'sub_index': f'{i}_{j}',
                     'lungs_bounding_box': lungs_bounding_box, 'radii': existing_radii, 'class': self._clazz})
                np.save(f'{OUTPUT_PATH}/{file_path}', img)
                print("image size: ",img.shape)
        return list_of_dicts

In [11]:
# RUN PREPROCESS
import pandas as pd
import numpy as np
from glob import glob
import os

annotations = pd.read_csv(RESOURCES_PATH + '/annotations.csv')
candidates = pd.read_csv(RESOURCES_PATH + '/candidates.csv')


def _get_positive_series():
    paths = glob(RESOURCES_PATH + '/*/' + "*.mhd")
    file_list = [f.split('/')[-1][:-4] for f in paths]
    series = annotations['seriesuid'].tolist()
    infected = [f for f in file_list if f in series]
    return infected


def _get_negative_series():
    paths = glob(RESOURCES_PATH + '/*/' + "*.mhd")
    file_list = [f.split('/')[-1][:-4] for f in paths]
    series = annotations['seriesuid'].tolist()
    cleans = [f for f in file_list if f not in series]
    return cleans


def save_preprocessed_data():
    [os.makedirs(d, exist_ok=True) for d in
     [f'{OUTPUT_PATH}/preprocessed/positives', f'{OUTPUT_PATH}/preprocessed/negatives']]
    meta_data = pd.DataFrame(columns=['seriesuid', 'spacing', 'lungs_bounding_box', 'centers', 'radii', 'class'])

    total_length_pos = len(_get_positive_series())
    total_length_neg = len(_get_negative_series())
    print("total_positive: ",total_length_pos)
    print("total_negative: ",total_length_neg)
    processed_num = 0
    neg_processed_num = 0

    i = 0
    for series_id in _get_positive_series():
        print("Processing id pos: ",series_id, " num-left: ", total_length_pos - processed_num)
        processed_num += 1

        nodule_coords_annot = annotations[annotations['seriesuid'] == series_id]
        tp_co = [(a['coordZ'], a['coordY'], a['coordX']) for a in nodule_coords_annot.iloc]
        radii = [(a['diameter_mm'] / 2) for a in nodule_coords_annot.iloc]
        ct = CTScan(seriesuid=series_id, centers=tp_co, radii=radii, clazz=1)
        ct.preprocess()
        ct.save_preprocessed_image()
        diction = ct.get_info_dict()
        print("diction: ",diction)
        meta_data.loc[len(meta_data)] = pd.Series(diction)
        # meta_data = pd.concat([meta_data, pd.Series(diction)], ignore_index=True)

    for series_id in _get_negative_series():
        print("Processing id neg: ",series_id, " num-left: ", total_length_neg - neg_processed_num)
        neg_processed_num += 1

        nodule_coords_candid = candidates[candidates['seriesuid'] == series_id]
        tp_co = [(a['coordZ'], a['coordY'], a['coordX']) for a in nodule_coords_candid.iloc]
        radii = list(np.random.randint(40, size=len(tp_co)))
        max_numbers_to_use = min(len(tp_co), 3)
        tp_co = tp_co[:max_numbers_to_use]
        radii = radii[:max_numbers_to_use]
        ct = CTScan(seriesuid=series_id, centers=tp_co, radii=radii, clazz=0)
        ct.preprocess()
        ct.save_preprocessed_image()
        diction = ct.get_info_dict()
        meta_data.loc[len(meta_data)] = pd.Series(diction)


    meta_data.to_csv(f'{OUTPUT_PATH}/preprocessed_meta.csv')


if __name__ == '__main__':
    save_preprocessed_data()

total_positive:  127
total_negative:  51
Processing id pos:  1.3.6.1.4.1.14519.5.2.1.6279.6001.107351566259572521472765997306  num-left:  127
diction:  {'seriesuid': '1.3.6.1.4.1.14519.5.2.1.6279.6001.107351566259572521472765997306', 'radii': [2.542202491, 2.822370585, 2.107226027, 4.3240046435, 3.6426707765, 3.0704480215], 'centers': [(109, 163, 271), (80, 159, 250), (96, 272, 102), (113, 258, 80), (117, 192, 41), (125, 256, 231)], 'spacing': [1.0, 1.0, 1.0], 'lungs_bounding_box': [(0, 91, 29), (259, 298, 281)], 'class': 1}
Processing id pos:  1.3.6.1.4.1.14519.5.2.1.6279.6001.114195693932194925962391697338  num-left:  126
diction:  {'seriesuid': '1.3.6.1.4.1.14519.5.2.1.6279.6001.114195693932194925962391697338', 'radii': [2.582524019], 'centers': [(165, 211, 262)], 'spacing': [0.9992163009404389, 1.0, 1.0], 'lungs_bounding_box': [(0, 63, 22), (319, 287, 298)], 'class': 1}
Processing id pos:  1.3.6.1.4.1.14519.5.2.1.6279.6001.115386642382564804180764325545  num-left:  125
diction:  {'

<ipython-input-9-ba1157a7f707>:278: UserWarning: Input image is entirely zero, no valid convex hull. Returning empty image
  lef = convex_hull_image(lef)


diction:  {'seriesuid': '1.3.6.1.4.1.14519.5.2.1.6279.6001.119806527488108718706404165837', 'radii': [3.000498944], 'centers': [(112, 99, 90)], 'spacing': [0.9989316239316239, 1.0, 1.0], 'lungs_bounding_box': [(0, 70, 26), (351, 318, 334)], 'class': 1}
Processing id pos:  1.3.6.1.4.1.14519.5.2.1.6279.6001.122914038048856168343065566972  num-left:  123
diction:  {'seriesuid': '1.3.6.1.4.1.14519.5.2.1.6279.6001.122914038048856168343065566972', 'radii': [3.480511817, 2.416944872, 2.525073948, 3.598196384, 2.9494190675, 2.998854362], 'centers': [(84, 256, 61), (91, 174, 109), (93, 251, 61), (104, 163, 58), (101, 244, 318), (119, 271, 278)], 'spacing': [1.0, 1.000000642475329, 1.000000642475329], 'lungs_bounding_box': [(0, 95, 41), (320, 327, 337)], 'class': 1}
Processing id pos:  1.3.6.1.4.1.14519.5.2.1.6279.6001.141430002307216644912805017227  num-left:  122
diction:  {'seriesuid': '1.3.6.1.4.1.14519.5.2.1.6279.6001.141430002307216644912805017227', 'radii': [2.358496372, 3.0247956435, 2.7

<ipython-input-9-ba1157a7f707>:278: UserWarning: Input image is entirely zero, no valid convex hull. Returning empty image
  lef = convex_hull_image(lef)
<ipython-input-9-ba1157a7f707>:277: UserWarning: Input image is entirely zero, no valid convex hull. Returning empty image
  rig = convex_hull_image(rig)
<ipython-input-9-ba1157a7f707>:318: UserWarning: Input image is entirely zero, no valid convex hull. Returning empty image
  return im, convex_hull_image(binary)


diction:  {'seriesuid': '1.3.6.1.4.1.14519.5.2.1.6279.6001.141511313712034597336182402384', 'radii': [8.29766746, 1.9522091525, 3.144089142], 'centers': [(237, 117, 239), (186, 236, 185), (256, 91, 99)], 'spacing': [0.9988670694864048, 1.0000003937752016, 1.0000003937752016], 'lungs_bounding_box': [(0, 57, 41), (326, 293, 279)], 'class': 1}
Processing id pos:  1.3.6.1.4.1.14519.5.2.1.6279.6001.142154819868944114554521645782  num-left:  120
diction:  {'seriesuid': '1.3.6.1.4.1.14519.5.2.1.6279.6001.142154819868944114554521645782', 'radii': [3.6516779], 'centers': [(262, 117, 182)], 'spacing': [1.0006410086002104, 1.0, 1.0], 'lungs_bounding_box': [(0, 29, 0), (312, 282, 282)], 'class': 1}
Processing id pos:  1.3.6.1.4.1.14519.5.2.1.6279.6001.146987333806092287055399155268  num-left:  119
diction:  {'seriesuid': '1.3.6.1.4.1.14519.5.2.1.6279.6001.146987333806092287055399155268', 'radii': [12.386639785], 'centers': [(122, 186, 301)], 'spacing': [0.998641304347826, 0.9999994187127976, 0.999

...

<ipython-input-9-ba1157a7f707>:278: UserWarning: Input image is entirely zero, no valid convex hull. Returning empty image
  lef = convex_hull_image(lef)


Processing id neg:  1.3.6.1.4.1.14519.5.2.1.6279.6001.619372068417051974713149104919  num-left:  4


<ipython-input-9-ba1157a7f707>:277: UserWarning: Input image is entirely zero, no valid convex hull. Returning empty image
  rig = convex_hull_image(rig)
<ipython-input-9-ba1157a7f707>:278: UserWarning: Input image is entirely zero, no valid convex hull. Returning empty image
  lef = convex_hull_image(lef)
<ipython-input-9-ba1157a7f707>:318: UserWarning: Input image is entirely zero, no valid convex hull. Returning empty image
  return im, convex_hull_image(binary)


Processing id neg:  1.3.6.1.4.1.14519.5.2.1.6279.6001.710845873679853791427022019413  num-left:  3
Processing id neg:  1.3.6.1.4.1.14519.5.2.1.6279.6001.965620538050807352935663552285  num-left:  2
Processing id neg:  1.3.6.1.4.1.14519.5.2.1.6279.6001.969607480572818589276327766720  num-left:  1


<ipython-input-9-ba1157a7f707>:278: UserWarning: Input image is entirely zero, no valid convex hull. Returning empty image
  lef = convex_hull_image(lef)


In [ ]:
# Load the first CSV file into a DataFrame
df1 = pd.read_csv('/content/drive/MyDrive/Preprocessed_Luna/preprocessed_meta.csv')

# Load the second CSV file into another DataFrame
df2 = pd.read_csv('/content/drive/MyDrive/Preprocessed_Luna_2/preprocessed_meta.csv')

# Merge the two DataFrames based on a common column (e.g., 'ID')
merged_df = pd.merge(df1, df2, on='ID', how='inner')  # Use 'how' parameter to specify the type of merge

# Alternatively, you can concatenate the DataFrames vertically if they have the same columns
# concatenated_df = pd.concat([df1, df2], ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('/content/drive/MyDrive/Preprocessed_Luna_2/preprocessed_meta_merged.csv', index=False)

In [ ]:
# RUN AUGMENTATION

import pandas as pd
import os
from ast import literal_eval


def _get_patches(record):
    rec = record
    seriesuid = rec['seriesuid']
    spacing = literal_eval(rec['spacing'])
    lungs_bounding_box = literal_eval(rec['lungs_bounding_box'])
    centers = literal_eval(rec['centers'])
    radii = literal_eval(rec['radii'])
    clazz = int(rec['class'])
    file_directory = 'preprocessed/positives' if clazz == 1 else 'preprocessed/negatives'
    file_path = f'{OUTPUT_PATH}/{file_directory}/{seriesuid}.npy'


    print("Processing: ", file_path)
    if (os.path.exists(file_path)):
      print("File exists!")
    else:
      print("File does not exist!")


    pm = PatchMaker(seriesuid=seriesuid, coords=centers, radii=radii, spacing=spacing,
                    lungs_bounding_box=lungs_bounding_box,
                    file_path=file_path, clazz=clazz)

    return pm.get_augmented_patches()


def save_augmented_data(preprocess_meta):
    [os.makedirs(d, exist_ok=True) for d in
     [f'{OUTPUT_PATH}/augmented/positives', f'{OUTPUT_PATH}/augmented/negatives']]
    augmentation_meta = pd.DataFrame(columns=['seriesuid',  'sub_index', 'centers', 'lungs_bounding_box', 'radii',
                                              'class'])
    list_of_positives = []
    list_of_negatives = []
    for rec in preprocess_meta.loc[preprocess_meta['class'] == 1].iloc:
        list_of_positives += _get_patches(rec)
    for rec in preprocess_meta.loc[preprocess_meta['class'] == 0].iloc:
        list_of_negatives += _get_patches(rec)
        # 33 percent of the data will be negative samples
        if len(list_of_negatives) > len(list_of_positives) / 2:
            break

    newRows = list_of_positives + list_of_negatives
    for row in newRows:
      augmentation_meta.loc[len(augmentation_meta)] = row

    # augmentation_meta = augmentation_meta.append(list_of_positives + list_of_negatives)
    augmentation_meta.to_csv(f'{OUTPUT_PATH}/augmented_meta.csv')
    print("Finished")


if __name__ == '__main__':
    p_meta = pd.read_csv(f'{OUTPUT_PATH}/preprocessed_meta.csv', index_col=0)
    save_augmented_data(p_meta)



Processing:  /content/drive/MyDrive/Preprocessed_Luna_2//preprocessed/positives/1.3.6.1.4.1.14519.5.2.1.6279.6001.104562737760173137525888934217.npy
File exists!
image size:  (128, 128, 128)
                                           seriesuid sub_index  \
0  1.3.6.1.4.1.14519.5.2.1.6279.6001.104562737760...       0_0   

          centers                    lungs_bounding_box                radii  \
0  [(82, 20, 73)]  [(-23, 111, 153), (263, -164, -263)]  [5.353915850020196]   

   class  
0      1  
Finished
